# **Data Preparation**

## **00. Import Libraries**

In [19]:
import pandas as pd
import numpy as np

## **01. Data Loading**

In [20]:
tasks = pd.read_csv('tasks.csv')
suppliers = pd.read_csv('suppliers.csv')
costs = pd.read_csv('cost.csv.zip')

In [21]:
tasks.set_index("Task ID", inplace = True)
print(f"There are {tasks.shape[0]} tasks and {tasks.shape[1]} features on the dataset:")
tasks.head()

There are 130 tasks and 116 features on the dataset:


,TF1,TF2,TF3,TF4,TF5,TF6,TF7,TF8,TF9,TF10,TF11,TF12,TF13,TF14,TF15,TF16,TF17,TF18,TF19,TF20,TF21,TF22,TF23,TF24,TF25,TF26,TF27,TF28,TF29,TF30,TF31,TF32,TF33,TF34,TF35,TF36,TF37,TF38,TF39,TF40,...,TF77,TF78,TF79,TF80,TF81,TF82,TF83,TF84,TF85,TF86,TF87,TF88,TF89,TF90,TF91,TF92,TF93,TF94,TF95,TF96,TF97,TF98,TF99,TF100,TF101,TF102,TF103,TF104,TF105,TF106,TF107,TF108,TF109,TF110,TF111,TF112,TF113,TF114,TF115,TF116
Task ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
T1,332,5123,647174452,2111,41.2063244192855%,140069227,22%,1438,0.280695,154237601,0.24,1216,0.237361,117762853,0.18,1299.17,203.39,-0.268212,2100,41.0,100.36,25.21,143,4525695.47,8379695.47,72229420,21187,35.825175,65.964066,397,1,10.160062,15.863304,75,1,468,50685398,0.09,0.08,4009,...,8838286.91,68607119,0,44,220.86,166.52,561,0,20338223.87,16463426.25,73331323,0,85.38,130.79,494,0,7632633.87,12022807.28,49415865,0,163.66,139.99,553,0,15088047.76,14958304.97,73331323,0,0.57,4.06,50,0,93828.91,759621.99,12264068,0,382,0.074566,28150864,0.04
T2,0,5792,327432022,1619,27.9523480662983%,105875589,32%,1097,0.189399,71537962,0.22,874,0.150898,51768065,0.16,1527.26,396.94,0.505234,2200,65.0,70.88,53.44,124,2640580.82,4955210.28,35757503,5188,46.709677,96.336451,820,1,14.683011,17.700759,80,1,390,34821718,0.07,0.11,3201,...,7433930.31,36767567,0,12,285.31,289.46,921,0,14626753.34,13448381.94,54133230,0,214.86,306.43,902,0,10136116.61,13629582.06,52238577,0,211.37,218.16,867,0,10488108.48,10218672.60,49219372,0,32.31,109.26,875,0,924983.41,3294170.58,29172807,0,552,0.095304,28894396,0.09
T3,469,4346,547810586,1376,31.6612977450529%,90478530,17%,1097,0.252416,122674168,0.22,834,0.191901,95043161,0.17,1345.70,218.69,-0.364256,2100,757.0,83.51,40.54,144,3804240.18,7891026.04,67729733,16821,30.180556,55.141097,311,1,9.727566,14.885829,75,1,328,35221874,0.08,0.06,2930,...,5973637.45,45018815,0,44,181.90,144.28,527,0,13134287.94,11332584.52,51132098,0,62.75,102.19,464,0,5044834.55,9352680.54,44732751,0,139.55,125.03,393,0,9743849.45,9122217.84,49212552,0,5.25,26.07,394,0,258487.96,1781016.69,39064840,0,706,0.162448,19579305,0.04
T4,529,7132,700169181,2362,33.1183398766124%,149526281,21%,1474,0.206674,155655199,0.22,1264,0.177229,119950245,0.17,1505.02,390.81,0.585402,2200,41.0,69.69,50.37,146,4795679.32,9312900.70,72209536,6668,48.849315,112.615571,1039,1,12.558890,14.664692,75,1,470,50030880,0.07,0.07,3932,...,8698222.69,66713288,0,51,357.24,362.71,1134,0,19814710.92,15549393.64,71540084,0,244.92,366.52,1114,0,10575493.30,13207831.29,45635766,0,261.69,267.44,988,0,14509089.56,13443310.09,71540084,0,23.70,89.43,997,0,640011.29,2434120.90,29710103,0,371,0.052019,22272333,0.03
T5,0,4767,278792402,1215,25.4877281308999%,87557248,31%,1038,0.217747,62825715,0.23,820,0.172016,48328267,0.17,1484.76,385.65,0.649069,2200,59.0,76.67,51.87,127,2195215.76,3973703.00,23271139,10738,37.535433,67.014781,556,1,12.580239,16.218148,80,1,340,27055689,0.07,0.10,2752,...,5418148.01,25093238,0,16,190.98,176.00,609,0,11100320.80,10305724.00,40795750,0,130.09,186.98,585,0,6991655.95,10192025.87,39366902,0,140.56,131.96,557,0,7976711.08,7688253.83,37041878,0,18.98,71.28,567,0,625549.93,2506860.98,22039095,0,93,0.019509,3058291,0.01


In [22]:
suppliers = suppliers.T
suppliers.columns = suppliers.iloc[0]
suppliers = suppliers.drop(suppliers.index[0])
suppliers.index.name = "Supplier ID"

print(f"There are {suppliers.shape[0]} suppliers and {suppliers.shape[1]} features on the dataset:")
suppliers.head()

There are 64 suppliers and 18 features on the dataset:


Features,SF1,SF2,SF3,SF4,SF5,SF6,SF7,SF8,SF9,SF10,SF11,SF12,SF13,SF14,SF15,SF16,SF17,SF18
Supplier ID,,,,,,,,,,,,,,,,,,
S1,100,1000,1000,50,20,10,2,80,2000,100,1000,5,1000,1000,500,5000,100,96
S2,100,1000,1000,50,20,10,0,80,2000,100,1000,5,1000,1000,500,5000,100,96
S3,100,1000,1000,50,20,10,2,80,2000,100,1000,5,1000,1000,500,5000,0,96
S4,100,1000,1000,50,20,10,2,80,2000,100,1000,5,1000,1000,500,5000,5000,96
S5,10,1000,100,500,200,10,2,80,200,100,2000,8,2000,100,2000,5000,15000,90


In [23]:
print(f"There are {costs.shape[0]} entries and {costs.shape[1]} columns on the costs dataset:")
costs.head()

There are 7680 entries and 3 columns on the costs dataset:


,Task ID,Supplier ID,Cost
0,T34,S1,0.478219
1,T34,S2,0.444543
2,T34,S3,0.521679
3,T34,S4,0.307331
4,T34,S5,0.357689


## **02. Cleaning and Preprocessing**

Identify Null Values

In [24]:
print(suppliers.isna().sum().sum())
print(tasks.isna().sum().sum())
print(costs.isna().sum().sum())

0
0
0


Identify Columns in Zero

In [25]:
supplier_columns = suppliers.sum(axis=0)[suppliers.sum(axis=0) == 0].count()
print(f"Suppliers dataset has {supplier_columns} supplier columns in zero")

tasks_columns = tasks.sum(axis=0)[tasks.sum(axis=0) == 0].count()
print(f"Tasks dataset has {tasks_columns} columns in zero")

cost_rows = costs[costs['Cost']==0].count().sum()
print(f"Costs dataset has {cost_rows} entries in zero")

Suppliers dataset has 0 supplier columns in zero
Tasks dataset has 10 columns in zero
Costs dataset has 0 entries in zero


Convert percentage strings to floats

In [26]:
tasks_clean = tasks.applymap(lambda x: float(x.strip('%')) / 100 if isinstance(x, str) and x.endswith('%') else x)
suppliers_clean = suppliers.applymap(lambda x: float(x.strip('%')) / 100 if isinstance(x, str) and x.endswith('%') else x)

Drop irrelevant or redundant columns

In [27]:
def drop_columns(dataset, dataset_name=""):
    num_columns_before = dataset.shape[1]
    dataset = dataset.loc[:, dataset.var(axis=0) != 0] # Drop columns with variance equal to 0
    dataset = dataset.loc[:, (dataset.sum(axis=0) != 0)] # Drop columns that sum 0
    num_columns_after = dataset.shape[1]
    num_columns_dropped = num_columns_before - num_columns_after
    print(f"Number of columns dropped from {dataset_name}: {num_columns_dropped}")
    return dataset

In [28]:
tasks_clean = drop_columns(tasks_clean, dataset_name='tasks_clean')
suppliers_clean = drop_columns(suppliers_clean, dataset_name='suppliers_clean')

Number of columns dropped from tasks_clean: 11
Number of columns dropped from suppliers_clean: 0


Elimination of highly correlated columns

In [29]:
def drop_corr_columns(dataset, dataset_name="", corr_treshold=0.95):
    num_columns_before = dataset.shape[1]
    
    corr_matrix = dataset.corr()
    high_corr = corr_matrix[(corr_matrix < -corr_treshold) | (corr_matrix > corr_treshold)].stack().reset_index()
    high_corr.columns = ['Feature1', 'Feature2', 'Correlation']
    high_corr = high_corr[high_corr['Feature1'] != high_corr['Feature2']]
    high_corr.drop_duplicates('Correlation', inplace = True)
    
    dataset = dataset.drop(columns=list(high_corr.Feature1.unique()))
    
    num_columns_after = dataset.shape[1]
    
    num_columns_dropped = num_columns_before - num_columns_after
    
    print(f"Number of columns dropped from {dataset_name}: {num_columns_dropped}")
    return dataset

In [30]:
tasks_clean = drop_corr_columns(tasks_clean, dataset_name="tasks_clean", corr_treshold=0.9)

Number of columns dropped from tasks_clean: 58


Normalize data [0,1]

In [31]:
def normalize_data_0_1(data):
    min_val = data.min()
    max_val = data.max()
    norm_data = (data - min_val) / (max_val - min_val)
    return norm_data

In [32]:
tasks_clean_norm = normalize_data_0_1(tasks_clean)
suppliers_clean_norm = normalize_data_0_1(suppliers_clean)

## **03. Analysing costs and eliminating the worst suppliers**

Select the number of top suppliers 

In [33]:
n = 59

Filter by only leaving the top-n suppliers by cost

In [34]:
costs_sorted = costs.sort_values(by=['Task ID', 'Cost'])
filtered_costs = costs_sorted.groupby('Task ID').head(n)

## **04. Create merged data**

In [35]:
merged_data = filtered_costs.merge(tasks_clean_norm, how='left', on='Task ID').merge(suppliers_clean_norm, how='left', on='Supplier ID').set_index('Task ID')
print(f"There are {merged_data.shape[0]} entries and {merged_data.shape[1]} columns on the merged dataset:")
merged_data.head()

There are 7080 entries and 67 columns on the merged dataset:


,Supplier ID,Cost,TF1,TF5,TF7,TF9,TF11,TF13,TF15,TF16,TF17,TF18,TF19,TF20,TF23,TF26,TF27,TF32,TF34,TF35,TF37,TF38,TF39,TF46,TF47,TF54,TF57,TF59,TF60,TF62,TF64,TF65,TF66,TF68,TF70,TF71,TF77,TF78,TF80,TF99,TF102,TF103,TF106,TF107,TF110,TF111,TF114,TF115,TF116,SF1,SF2,SF3,SF4,SF5,SF6,SF7,SF8,SF9,SF10,SF11,SF12,SF13,SF14,SF15,SF16,SF17,SF18
Task ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
T1,S18,0.355652,0.470255,0.537396,0.278689,0.459535,0.4,0.867832,0.666667,0.094796,0.127437,0.229699,0.831366,0.0,0.765217,1.0,0.270607,0.207406,0.372093,1.0,0.772459,0.571429,0.222222,0.461538,0.466667,0.650486,0.820513,0.574468,0.615176,0.905647,0.592593,0.758729,0.515152,0.00002,0.000022,0.51938,0.410336,1.0,0.333333,0.289756,0.340138,0.501986,0.012123,0.040193,0.086781,0.1754,0.198613,0.469404,0.333333,0.000000,1.0,0.000000,0.090909,1.000000,0.090909,0.666667,0.0,1.000000,0.183673,1.000000,1.000000,0.000000,0.473684,0.230769,0.0,1.00,1.00
T1,S56,0.356349,0.470255,0.537396,0.278689,0.459535,0.4,0.867832,0.666667,0.094796,0.127437,0.229699,0.831366,0.0,0.765217,1.0,0.270607,0.207406,0.372093,1.0,0.772459,0.571429,0.222222,0.461538,0.466667,0.650486,0.820513,0.574468,0.615176,0.905647,0.592593,0.758729,0.515152,0.00002,0.000022,0.51938,0.410336,1.0,0.333333,0.289756,0.340138,0.501986,0.012123,0.040193,0.086781,0.1754,0.198613,0.469404,0.333333,0.000000,0.0,1.000000,0.090909,0.090909,0.090909,1.000000,1.0,1.000000,0.000000,0.000000,0.571429,0.473684,1.000000,0.230769,0.0,0.15,1.00
T1,S47,0.357635,0.470255,0.537396,0.278689,0.459535,0.4,0.867832,0.666667,0.094796,0.127437,0.229699,0.831366,0.0,0.765217,1.0,0.270607,0.207406,0.372093,1.0,0.772459,0.571429,0.222222,0.461538,0.466667,0.650486,0.820513,0.574468,0.615176,0.905647,0.592593,0.758729,0.515152,0.00002,0.000022,0.51938,0.410336,1.0,0.333333,0.289756,0.340138,0.501986,0.012123,0.040193,0.086781,0.1754,0.198613,0.469404,0.333333,0.000000,0.0,0.473684,0.000000,0.000000,0.000000,0.666667,1.0,0.000000,0.183673,0.473684,0.000000,0.000000,0.000000,0.000000,0.6,1.00,0.75
T1,S27,0.367182,0.470255,0.537396,0.278689,0.459535,0.4,0.867832,0.666667,0.094796,0.127437,0.229699,0.831366,0.0,0.765217,1.0,0.270607,0.207406,0.372093,1.0,0.772459,0.571429,0.222222,0.461538,0.466667,0.650486,0.820513,0.574468,0.615176,0.905647,0.592593,0.758729,0.515152,0.00002,0.000022,0.51938,0.410336,1.0,0.333333,0.289756,0.340138,0.501986,0.012123,0.040193,0.086781,0.1754,0.198613,0.469404,0.333333,0.090909,1.0,0.473684,0.090909,0.000000,0.090909,1.000000,1.0,0.473684,0.000000,0.000000,1.000000,0.473684,0.473684,0.000000,1.0,0.02,0.75
T1,S34,0.367398,0.470255,0.537396,0.278689,0.459535,0.4,0.867832,0.666667,0.094796,0.127437,0.229699,0.831366,0.0,0.765217,1.0,0.270607,0.207406,0.372093,1.0,0.772459,0.571429,0.222222,0.461538,0.466667,0.650486,0.820513,0.574468,0.615176,0.905647,0.592593,0.758729,0.515152,0.00002,0.000022,0.51938,0.410336,1.0,0.333333,0.289756,0.340138,0.501986,0.012123,0.040193,0.086781,0.1754,0.198613,0.469404,0.333333,0.000000,1.0,0.473684,1.000000,0.000000,1.000000,0.333333,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.473684,1.000000,0.6,0.30,0.75


## **05. Save CSV Files**

In [36]:
# Print name and shape of each dataset
print("tasks_clean:", tasks_clean.shape)
print("suppliers_clean:", suppliers_clean.shape)
print("merged_data:", merged_data.shape)
print("filtered_costs:", filtered_costs.shape)
print("tasks_clean_norm:", tasks_clean_norm.shape)
print("suppliers_clean_norm:", suppliers_clean_norm.shape)

# Save datasets to CSV
tasks_clean.to_csv("tasks_clean.csv")
suppliers_clean.to_csv("suppliers_clean.csv")
merged_data.to_csv("merged_tasks_suppliers.csv")
filtered_costs.to_csv("filtered_costs.csv")
tasks_clean_norm.to_csv("tasks_norm.csv")
suppliers_clean_norm.to_csv("suppliers_norm.csv")

tasks_clean: (130, 47)
suppliers_clean: (64, 18)
merged_data: (7080, 67)
filtered_costs: (7080, 3)
tasks_clean_norm: (130, 47)
suppliers_clean_norm: (64, 18)
